In [1]:
import re
from collections import namedtuple, deque
from functools import reduce

In [2]:
Ingredients = namedtuple('Ingredients', 'ore,clay,obsidian')
Blueprint = namedtuple('Blueprint', 'ore_robot,clay_robot,obsidian_robot,geode_robot')
Node = namedtuple('Node', 'time,ore,clay,obsidian,geode,ore_robot,clay_robot,obsidian_robot,geode_robot')

In [3]:
def get_max_geode(blueprint, time=23, prune=10000):
    start = Node(0, 0, 0, 0, 0, 1, 0, 0, 0)

    queue = deque()
    queue.append(start)

    max_geode = 0

    while len(queue) > 0:
        if len(queue) > prune:
            sorted_queue = sorted(queue,
                key=lambda x: [
    #                 x.time,
                    -x.geode, -x.geode_robot,
                    -x.obsidian, -x.obsidian_robot,
                    -x.clay, -x.clay_robot,
                    -x.ore, -x.ore_robot
                ]
            )
            queue = deque(sorted_queue[:prune])

        curr = queue.popleft()

        current_ore      = curr.ore
        current_clay     = curr.clay
        current_obsidian = curr.obsidian
        current_geode    = curr.geode

        current_ore      += curr.ore_robot
        current_clay     += curr.clay_robot
        current_obsidian += curr.obsidian_robot
        current_geode    += curr.geode_robot

        if curr.time == time:
            if current_geode > max_geode:
                max_geode = current_geode
            continue

        can_build = 0

        if curr.ore >= blueprint.ore_robot.ore:
            if curr.ore_robot < max(blueprint, key=lambda x: x.ore).ore:
                queue.append(
                    Node(
                        curr.time+1, 
                        current_ore-blueprint.ore_robot.ore, 
                        current_clay-blueprint.ore_robot.clay, 
                        current_obsidian-blueprint.ore_robot.obsidian, 
                        current_geode, 
                        curr.ore_robot+1, curr.clay_robot, curr.obsidian_robot, curr.geode_robot
                    )
                )
                can_build += 1

        if curr.ore >= blueprint.clay_robot.ore:
            if curr.clay_robot < max(blueprint, key=lambda x: x.clay).clay:
                queue.append(
                    Node(
                        curr.time+1, 
                        current_ore-blueprint.clay_robot.ore, 
                        current_clay-blueprint.clay_robot.clay,
                        current_obsidian-blueprint.clay_robot.obsidian, 
                        current_geode, 
                        curr.ore_robot, curr.clay_robot+1, curr.obsidian_robot, curr.geode_robot
                    )
                )
                can_build += 1

        if curr.ore >= blueprint.obsidian_robot.ore and curr.clay >= blueprint.obsidian_robot.clay:
            if curr.obsidian_robot < max(blueprint, key=lambda x: x.obsidian).obsidian:
                queue.append(
                    Node(
                        curr.time+1, 
                        current_ore-blueprint.obsidian_robot.ore, 
                        current_clay-blueprint.obsidian_robot.clay, 
                        current_obsidian-blueprint.obsidian_robot.obsidian, 
                        current_geode, 
                        curr.ore_robot, curr.clay_robot, curr.obsidian_robot+1, curr.geode_robot
                    )
                )
                can_build += 1

        if curr.ore >= blueprint.geode_robot.ore and curr.obsidian >= blueprint.geode_robot.obsidian:        
            queue.append(
                Node(
                    curr.time+1, 
                    current_ore-blueprint.geode_robot.ore, 
                    current_clay-blueprint.geode_robot.clay, 
                    current_obsidian-blueprint.geode_robot.obsidian, 
                    current_geode, 
                    curr.ore_robot, curr.clay_robot, curr.obsidian_robot, curr.geode_robot+1
                )
            )
            can_build += 1

        if can_build < 4:
            queue.append(
                Node(
                    curr.time+1, current_ore, current_clay, current_obsidian, current_geode, 
                    curr.ore_robot, curr.clay_robot, curr.obsidian_robot, curr.geode_robot
                )
            )
            
    return max_geode

In [4]:
with open('input.txt') as f:
    data = f.read().rstrip()

## part 1

In [5]:
scores = {}

for line in data.splitlines():
    b, r1, r2, r3, r4, r5, r6 = list(map(int, re.findall('\d+', line)))
    blueprint = Blueprint(
        Ingredients(r1, 0, 0), Ingredients(r2, 0, 0),
        Ingredients(r3, r4, 0), Ingredients(r5, 0, r6)
    )
    scores[b] = get_max_geode(blueprint, prune=3000)
    
print(scores)
print(f"answer: {sum(k * v for k, v in scores.items())}")

{1: 4, 2: 6, 3: 3, 4: 0, 5: 5, 6: 15, 7: 12, 8: 8, 9: 0, 10: 2, 11: 5, 12: 5, 13: 0, 14: 0, 15: 3, 16: 2, 17: 0, 18: 1, 19: 3, 20: 0, 21: 0, 22: 0, 23: 2, 24: 3, 25: 2, 26: 0, 27: 0, 28: 8, 29: 0, 30: 7}
answer: 1177


## part 2

In [6]:
scores = {}

for line in data.splitlines()[:3]:
    b, r1, r2, r3, r4, r5, r6 = list(map(int, re.findall('\d+', line)))
    blueprint = Blueprint(
        Ingredients(r1, 0, 0), Ingredients(r2, 0, 0),
        Ingredients(r3, r4, 0), Ingredients(r5, 0, r6)
    )
    scores[b] = get_max_geode(blueprint, time=31, prune=10000)
    
print(scores)
print(f"answer: {reduce((lambda x, y: x * y), scores.values())}")

{1: 44, 2: 46, 3: 31}
answer: 62744
